In [1]:
%pip install requests python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

# Load API key
load_dotenv("env.template")
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

def get_stock_data(symbol: str):
    """Fetch daily stock data from Alpha Vantage API."""
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "apikey": API_KEY,
        "outputsize": "full"
    }
    r = requests.get(url, params=params)
    data = r.json()["Time Series (Daily)"]

    df = pd.DataFrame.from_dict(data, orient="index", dtype=float)
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    df = df.rename(columns={
        "1. open": "Open",
        "2. high": "High",
        "3. low": "Low",
        "4. close": "Close",
        "5. adjusted close": "Adj Close",
        "6. volume": "Volume"
    })
    return df

# Example usage
aapl = get_stock_data("AAPL")
nvda = get_stock_data("NVDA")
lyft = get_stock_data("LYFT")


KeyError: 'Time Series (Daily)'

In [ ]:
def add_ema_dema(df, span=20):
    """Add EMA and DEMA columns to the DataFrame."""
    df[f"EMA_{span}"] = df["Close"].ewm(span=span, adjust=False).mean()
    ema = df[f"EMA_{span}"]
    df[f"DEMA_{span}"] = 2*ema - ema.ewm(span=span, adjust=False).mean()
    return df

# Apply to all stocks
aapl = add_ema_dema(aapl, 20)
nvda = add_ema_dema(nvda, 20)
lyft = add_ema_dema(lyft, 20)


In [ ]:
import matplotlib.pyplot as plt

def plot_stock(df, symbol, span=20):
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df["Close"], label="Close Price", alpha=0.6)
    plt.plot(df.index, df[f"EMA_{span}"], label=f"EMA {span}", linestyle="--")
    plt.plot(df.index, df[f"DEMA_{span}"], label=f"DEMA {span}", linestyle=":")
    plt.title(f"{symbol} Stock with EMA & DEMA")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

plot_stock(aapl, "AAPL")
plot_stock(nvda, "NVDA")
plot_stock(lyft, "LYFT")


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

def decompose_series(df, symbol):
    decomposition = seasonal_decompose(df["Adj Close"], model="multiplicative", period=252)
    fig = decomposition.plot()
    fig.suptitle(f"{symbol} - Time Series Decomposition", fontsize=14)
    plt.show()

decompose_series(aapl, "AAPL")
decompose_series(nvda, "NVDA")
decompose_series(lyft, "LYFT")
